<h1>Data Visualization</h1>

<p style="font-size: 16px">In this notebook I'm assuming that you have already followed the previous one and signed up to Google Earth Engine and Authenticated you account. So the goal on this notebook is to dive deeper into:</p>
<ul>
    <li>Google Earth Engine structure</li>
    <li>Google's Data Catalog</li>
    <li>Remote Sensing data acquisition</li>
    <li>Image visualization with folium and geehydro</li>
    <li>Filter data</li>    
</ul>

In [ ]:
import ee
import folium
import geehydro
from IPython.display import Image

In [ ]:
ee.Initialize()

<p style="font-size: 16px">First we have to initialize our Folium map.Here we are setting two basic parameters: location, that is where the center of the map will be located when the map is initialized; zoom_start, that is the zoom level that the map will have when initialize. If you press shift+tab inside the constructor of the function you will see many other parameters that have very intuitive names that you can play with.
After that, we use the setOptions function, that is from geehydro library. geehydro is a package built that implements several methods for creating an interaction between folium and Earth Engine, with it we can add layers to the map and set some controls to it. With setOptions function we are setting the base layer of our map, and we have the following options:</p>
<ul>
    <li>"ROADMAP"
        <ul>
            <li>It's a Google Maps-like map, with road information (obviously)</li>
            <li>It also has information about countries and states' boundaries</li>
        </ul>
    </li>
    <li>"SATELLITE"
        <ul>
            <li>It has satellite image, with no information about countries, boundaries, etc. It's only images</li>
            <li>Images can vary on quality from place to place</li>
        </ul>
    </li>
    <li>"HYBRID"
        <ul>
            <li>It's a mix of SATELLITE and ROADMAP</li>
            <li>You have satellite images with boundaries, roads, states, etc.</li>
        </ul>
    </li>
    <li>"TERRAIN"
        <ul>
            <li>Similar to ROADMAP with soil details</li>
            <li>You can visualize mountains and types of soil</li>
        </ul>
    </li> 
    <li>"NAIP"
        <ul>
            <li>Looks like it's in development yet</li>
            <li>NAIP stands for National Agriculture Imagery Program from US</li>
        </ul>
    </li>
    <li>"ESRI"
        <ul>
            <li>Diffenrent satellite visualization</li>
            <li>Higher resolution images than "SATELLITE"</li>
            <li>In some areas, if you try to zoom in, it might not have the data available</li>
        </ul>
    </li>
</ul>

<p style="font-size: 16px">We can also personalize our map in other ways that we will see further in this series of notebooks. By now, this is enough.</p>

In [ ]:
# Initializing the folium map
_map = folium.Map(location=[-0.784176, -60.084085], zoom_start=6)
# Setting the type of map we want to have on background
_map.setOptions('ROADMAP')

<h2>Data Catalog</h2>

<p>The Earth Engine API has its own data catalog and Google is doing a great job on providing it in an easy to use and free way. You can find all the data available for Earth Engine in this <a href="https://developers.google.com/earth-engine/datasets">link</a>. In this series we will be working mainly with raw data, e.g. bands of satellite imagery, as our goal is to apply some Machine Learning and Deep Learning techniques using Remote Sensing data.

In the EE's data catalog we can find imagery from Landsat series (1, 4, 5, 7 and 8), sentinel series (1, 2, 3 and 5P), MODIS (too many to put here), Planet labs SkySat and NAIP. Those are very precious data that we will be exploring in details set-by-set. It's very easy to acquire the data using EE, you just need to access the data catalog, search for the dataset you want to use, and in the page of this dataset there will be an Id similar to this: LANDSAT/LC08/C01/T1. Then, you just need to paste it into the ee.ImageCollection() function, that we will be seing in the folling cells.
    
Now we will be taking a closer look at Landsat imagery, more specificly at Landsat-8 imagery. In the <a href="https://developers.google.com/earth-engine/datasets/catalog/landsat-8">Landsat-8 section</a> we find 3 options: Surface Reflectance, Top of Atmosphere and Raw Images. We will understand better each of these sections further, for now let's just say that there are raw images with a lot of noise which need some preprocessing, and top of atmosphere and surface reflectance are images after some preprocessing methods are applied. In this notebook, we will visualize the raw images that have the ID LANDSAT/LC08/C01/T1. This ID is from a ImageCollection, a class that its object contains many images and methods to filter and process those images.
    
In this catalog there are some Feature Collections also. We can find a lot of them with different purposes and we will be using some of them in this series. As an example we gonna use the <a href="https://developers.google.com/earth-engine/datasets/catalog/USDOS_LSIB_SIMPLE_2017">Large Scale International Boundary Polygons, Simplified (LSIB)</a> FeatureCollection to filter images that cover only a country's boundaries. In the data catalog you can find the table schema to understand better how to use the FeatureCollection. Similar to ImageCollection, FeatureCollection is a class that its object has many features, and those features can be countries boundaries, states boundaries, U.S terrestrial and marine protected areas boundaries, global land ice measurements, and many more.
    
So what we gonna do in the next cells is load images from Landsat-8 that cover Brazil's territory. Those images will be a 1-year composite, i.e. we select an image collection from a 1-year time range and select the median value for each pixel, generating only one image. Now let's go to the code
</p>



<h2>Let's Code</h2>

<p>First, we have to select the FeatureCollection that contains boundaries from all countries. As said before, the ID of this feature collection can be found on the Earth Engine Data Catalog.
    
So, we use the ee.FeatureCollection constructor and the ID to get those boundaries</p>

In [ ]:
# Getting the Large Scale International Boundary Polygons FeatureCollection, i.e. getting the boundaries of all countries in the world 
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')

However, we just want to get Brazil's boundaries and the above cell returned to us boundaries from the whole world. Now what we have to do is use a filter to get what we want. An object of the FeatureCollection class has a method called filter, which receives a filter from the Filter class, and this filter filters based on the metadata of the FeatureCollection object. So we are using the ee.Filter.eq() function that filters to metadata equal to the given value (eq stands for equal). 'country_na' and 'Brazil' are atributes of the FeatureCollection table schema, 'country_na' stands for Country Name, and 'Brazil' stands for the name of the country that we want to get the boundaries, so for filtering we have to first select the way we want to access it, in this case by country name and then we have to select the country that we want. There are some other options in the way to access this data.

Based on the LSIB's table schema, we have the following options to use as filter:

<ul>
    <li>Country Code
        <ul>
            <li>Using the string 'country_co' on the filter</li>
            <li>You can access countries boundaries using their official code</li>
            <li>ee.Filter.eq('country_co', 'BR')</li>
        </ul>
    </li>
    <li>Country Name
        <ul>
            <li>Using the string 'country_na' on the filter</li>
            <li>You can access countries boundaries using the name</li>
            <li>ee.Filter.eq('country_na', 'Brazil')</li>
        </ul>
    </li>
    <li>World Region
        <ul>
            <li>Using the string 'wld_rgn'</li>
            <li>You can access boundaries of different continents</li>
            <li>ee.Filter.eq('wld_rgn', 'South America')</li>
        </ul>
</ul>

Some regions are too big or have too much polygons and load their data in an interactive map is not a good idea, if you try it will raise an error.

In [ ]:
# Filtering to Brazil's boundaries
bound_brazil = countries.filter(ee.Filter.eq('country_na', 'Brazil'))

In [ ]:
# Filtering landsat-8 iamgery by date and by location using the boundaries we selected
dataset = ee.ImageCollection('LANDSAT/LC08/C01/T1').filterDate('2017-01-01', '2017-12-31').filterBounds(bound_brazil)

In [ ]:
# Selecting the bands we are going to use
true_color = dataset.select(['B4', 'B3', 'B2'])
# Defining the visualization parameters to the interactive map
true_color_vis = {
    'min' : 0.0,
    'max' : 30000.0,
}

In [ ]:
# Adding the landsat-8' images layer
_map.addLayer(true_color.median(), true_color_vis, 'True Color')
# Adding a layer to visualize the FeatureCollection
_map.addLayer(bound_brazil, {'color': 'red'}, 'FeatureCollection')
# Adding a layer to visualize only the bondaries
_map.addLayer(ee.Image().paint(bound_brazil, color=0, width=3), {'palette' : '0000FF'}, 'Brazil')

In [ ]:
# Setting folium's map control with geehydro
_map.setControlVisibility()

In [ ]:
# Show the map
_map